IA3配条代码参考自：

[peft_ia3_mindnlp.ipynb](https://github.com/mindspore-lab/mindnlp/tree/master/llm/peft/ia3/seq_2_seq)

模型配置参考：基于MindSpore NLP的Roberta模型Prompt Tuning

https://blog.csdn.net/Kenji_Shinji/article/details/144395136

In [1]:
import os
import mindspore
from mindnlp.transformers import AutoModelForSequenceClassification, AutoTokenizer
from mindnlp.peft import get_peft_config, get_peft_model, get_peft_model_state_dict, LoraConfig, TaskType,IA3Config
from mindnlp.dataset import load_dataset
from mindnlp.core import ops
from mindnlp.common.optimization import get_linear_schedule_with_warmup
from tqdm import tqdm
from mindnlp import evaluate

model_name_or_path = "roberta-large"
tokenizer_name_or_path = "roberta-large"

# checkpoint_name = "financial_sentiment_analysis_lora_v1.ckpt"
checkpoint_name = "RoBERTa_IA3_v1.ckpt"

# 优化训练表现可以调整以下数值
max_length = 128
lr = 3e-5
num_epochs = 5
batch_size = 8

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib

In [2]:
# creating model
peft_config = IA3Config(task_type=TaskType.SEQ_CLS, inference_mode=False) #TaskType从SEQ_2_SEQ_LM修改为SEQ_CLS
#TaskType.SEQ_2_SEQ_LM 是专门为序列到序列生成任务（如机器翻译、文本摘要等）设计的。对于 RoBERTa 模型和 AutoModelForSequenceClassification，你应该将 task_type 设置为 TaskType.SEQ_CLS，因为这是针对序列分类任务的正确配置。
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, return_dict=True) #对应修改为AutoModelForSequenceClassification
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,223,682 || all params: 356,585,476 || trainable%: 0.34316652874555104


In [3]:
#加载tokenizer
if any(k in model_name_or_path for k in ("gpt", "opt", "bloom")):
    padding_side = "left"
else:
    padding_side = "right"
 
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side=padding_side)
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted, and will be then set to `False` by default. 
  warnings.warn(


In [4]:
# mindspore.dataset.config.set_seed(123)
# loading dataset
# dataset = load_dataset("financial_phrasebank", "sentences_allagree")
datasets = load_dataset("glue", "mrpc")
print(next(datasets['train'].create_dict_iterator()))

{'sentence1': Tensor(shape=[], dtype=String, value= 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .'), 'sentence2': Tensor(shape=[], dtype=String, value= 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'), 'label': Tensor(shape=[], dtype=Int64, value= 1), 'idx': Tensor(shape=[], dtype=Int64, value= 0)}


In [5]:
from mindnlp.dataset import BaseMapFunction
 
class MapFunc(BaseMapFunction):
    def __call__(self, sentence1, sentence2, label, idx):
        outputs = tokenizer(sentence1, sentence2, truncation=True, max_length=None)
        return outputs['input_ids'], outputs['attention_mask'], label
 
def get_dataset(dataset, tokenizer):
    input_colums=['sentence1', 'sentence2', 'label', 'idx']
    output_columns=['input_ids', 'attention_mask', 'labels']
    dataset = dataset.map(MapFunc(input_colums, output_columns),
                          input_colums, output_columns)
    dataset = dataset.padded_batch(batch_size, pad_info={'input_ids': (None, tokenizer.pad_token_id),
                                                         'attention_mask': (None, 0)})
    return dataset
 
train_dataset = get_dataset(datasets['train'], tokenizer)
eval_dataset = get_dataset(datasets['validation'], tokenizer)

In [6]:
print(next(train_dataset.create_dict_iterator()))

{'input_ids': Tensor(shape=[8, 67], dtype=Int64, value=
[[    0, 10127,  1001 ...     1,     1,     1],
 [    0,   975, 26802 ...     1,     1,     1],
 [    0,  1213,    56 ...     1,     1,     1],
 ...
 [    0,  9064, 32497 ...     1,     1,     1],
 [    0,   133,  4417 ...     1,     1,     1],
 [    0,   133, 19888 ...     1,     1,     1]]), 'attention_mask': Tensor(shape=[8, 67], dtype=Int64, value=
[[1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 ...
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0]]), 'labels': Tensor(shape=[8], dtype=Int64, value= [1, 0, 1, 0, 1, 1, 0, 1])}


In [7]:
metric = evaluate.load("glue", "mrpc")

In [8]:
from mindnlp.core import optim
# optimizer and lr scheduler
optimizer = optim.AdamW(model.trainable_params(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataset) * num_epochs),
)

In [ ]:
from mindnlp.core import value_and_grad
# training and evaluation

def forward_fn(**batch):
    outputs = model(**batch)
    loss = outputs.loss
    return loss

# 使用value_and_grad装饰前向传播函数，以便在训练时获取损失值和梯度
grad_fn = value_and_grad(forward_fn, model.trainable_params())

# 开始训练和评估循环
for epoch in range(num_epochs):
    # 设置模型为训练模式
    model.set_train()
    total_loss = 0
    train_total_size = train_dataset.get_dataset_size()
    # 遍历训练数据集
    for step, batch in enumerate(tqdm(train_dataset.create_dict_iterator(), total=train_total_size)):
        # 清除上一步的梯度
        optimizer.zero_grad()
        # 计算损失值和梯度
        loss = grad_fn(**batch)
         # 更新模型参数
        optimizer.step()
        # 累加损失值
        total_loss += loss.float() #这一步不一样
        # 更新学习率调度器
        lr_scheduler.step()

    # 设置模型为评估模式
    model.set_train(False)
    eval_loss = 0 #这一步没有
    eval_preds = [] #这一步没有
    eval_total_size = eval_dataset.get_dataset_size()
    # 遍历评估数据集
    for step, batch in enumerate(tqdm(eval_dataset.create_dict_iterator(), total=eval_total_size)):
        # 禁用梯度计算，进行前向传播
        with mindspore._no_grad(): # 没有这句
            outputs = model(**batch)
        loss = outputs.loss        # 以下都不一样
        eval_loss += loss.float()
         # 将模型预测结果解码为文本并存储
        #eval_preds.extend(
        #    tokenizer.batch_decode(ops.argmax(outputs.logits, -1).asnumpy(), skip_special_tokens=True)
        #)

        # 对于序列分类任务，直接获取类别索引
        print(f"{ops.argmax(outputs.logits, -1)=}: {ops.argmax(outputs.logits, -1).asnumpy()=} {ops.argmax(outputs.logits, -1).asnumpy().tolist()=} ")
        eval_preds.extend(ops.argmax(outputs.logits, -1).asnumpy().tolist())
    
# 计算评估期损失
    eval_epoch_loss = eval_loss / len(eval_dataset)
    eval_ppl = ops.exp(eval_epoch_loss)
# 计算训练期损失
    train_epoch_loss = total_loss / len(train_dataset)
    train_ppl = ops.exp(train_epoch_loss)
# 打印训练和评估结果
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

  2%|▏         | 1/51 [00:00<00:15,  3.20it/s]

ops.argmax(outputs.logits, -1)=Tensor(shape=[8], dtype=Int64, value= [1, 1, 1, 1, 1, 1, 1, 1]): ops.argmax(outputs.logits, -1).asnumpy()=array([1, 1, 1, 1, 1, 1, 1, 1], dtype=int64) ops.argmax(outputs.logits, -1).asnumpy().tolist()=[1, 1, 1, 1, 1, 1, 1, 1] 


  4%|▍         | 2/51 [00:00<00:13,  3.56it/s]

ops.argmax(outputs.logits, -1)=Tensor(shape=[8], dtype=Int64, value= [1, 1, 1, 1, 1, 1, 1, 1]): ops.argmax(outputs.logits, -1).asnumpy()=array([1, 1, 1, 1, 1, 1, 1, 1], dtype=int64) ops.argmax(outputs.logits, -1).asnumpy().tolist()=[1, 1, 1, 1, 1, 1, 1, 1] 


  8%|▊         | 4/51 [00:00<00:10,  4.50it/s]

...

 49%|████▉     | 25/51 [00:02<00:02, 11.13it/s]

ops.argmax(outputs.logits, -1)=Tensor(shape=[8], dtype=Int64, value= [1, 1, 1, 1, 1, 1, 1, 1]): ops.argmax(outputs.logits, -1).asnumpy()=array([1, 1, 1, 1, 1, 1, 1, 1], dtype=int64) ops.argmax(outputs.logits, -1).asnumpy().tolist()=[1, 1, 1, 1, 1, 1, 1, 1] 
ops.argmax(outputs.logits, -1)=Tensor(shape=[8], dtype=Int64, value= [1, 1, 1, 1, 1, 1, 1, 1]): ops.argmax(outputs.logits, -1).asnumpy()=array([1, 1, 1, 1, 1, 1, 1, 1], dtype=int64) ops.argmax(outputs.logits, -1).asnumpy().tolist()=[1, 1, 1, 1, 1, 1, 1, 1] 
ops.argmax(outputs.logits, -1)=Tensor(shape=[8], dtype=Int64, value= [1, 1, 1, 1, 1, 1, 1, 1]): ops.argmax(outputs.logits, -1).asnumpy()=array([1, 1, 1, 1, 1, 1, 1, 1], dtype=int64) ops.argmax(outputs.logits, -1).asnumpy().tolist()=[1, 1, 1, 1, 1, 1, 1, 1] 


 57%|█████▋    | 29/51 [00:03<00:01, 11.23it/s]

ops.argmax(outputs.logits, -1)=Tensor(shape=[8], dtype=Int64, value= [1, 1, 1, 1, 1, 1, 1, 1]): ops.argmax(outputs.logits, -1).asnumpy()=array([1, 1, 1, 1, 1, 1, 1, 1], dtype=int64) ops.argmax(outputs.logits, -1).asnumpy().tolist()=[1, 1, 1, 1, 1, 1, 1, 1] 
ops.argmax(outputs.logits, -1)=Tensor(shape=[8], dtype=Int64, value= [1, 1, 1, 1, 1, 1, 1, 1]): ops.argmax(outputs.logits, -1).asnumpy()=array([1, 1, 1, 1, 1, 1, 1, 1], dtype=int64) ops.argmax(outputs.logits, -1).asnumpy().tolist()=[1, 1, 1, 1, 1, 1, 1, 1] 
ops.argmax(outputs.logits, -1)=Tensor(shape=[8], dtype=Int64, value= [1, 1, 1, 1, 1, 1, 1, 1]): ops.argmax(outputs.logits, -1).asnumpy()=array([1, 1, 1, 1, 1, 1, 1, 1], dtype=int64) ops.argmax(outputs.logits, -1).asnumpy().tolist()=[1, 1, 1, 1, 1, 1, 1, 1] 


 61%|██████    | 31/51 [00:03<00:01, 11.26it/s]

In [ ]:
print(f"{eval_preds=}")

In [ ]:
# 打印前5个样本的logits
print(f"Sample logits: {outputs.logits[:100]}")

In [ ]:
# print accuracy
# 初始化正确预测和总样本的计数器
correct = 0
total = 0

# 初始化用于存储真实标签的列表
ground_truth = []

# 遍历预测结果和验证数据集
for pred, data in zip(eval_preds, datasets['validation'].create_dict_iterator(output_numpy=True)):
    # 获取真实的文本标签
    true = str(data['label'])
    # 将真实标签添加到ground_truth列表中
    ground_truth.append(true)
    # 如果预测的标签与真实标签一致，则正确计数器加一
    if pred.strip() == true.strip():
        correct += 1
    # 总样本计数器加一
    total += 1
# 计算准确率
accuracy = correct / total * 100
# 输出准确率
print(f"{accuracy=} % on the evaluation dataset")
# 输出前10个预测结果
print(f"{eval_preds[:10]=}")
# 输出前10个真实标签
print(f"{ground_truth[:10]=}")

In [ ]:
datasets

In [ ]:
# 获取数据集迭代器的下一个元素
next(eval_dataset.create_tuple_iterator())

In [ ]:
# saving model
# 构建PEFT模型的唯一标识符
# 该标识符结合了模型名称或路径、PEFT配置的类型和任务类型，用于区分不同的PEFT模型
peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"

# 保存预训练的PEFT模型
# 使用构建的PEFT模型标识符作为模型ID，将模型保存到指定目录或路径
model.save_pretrained(peft_model_id)

In [ ]:
# 构建checkpoint文件的路径
ckpt = f"{peft_model_id}/adapter_model.ckpt"
# 使用shell命令检查checkpoint文件的大小
!du -h $ckpt

In [ ]:
# 导入Peft模型和配置相关的模块
from mindnlp.peft import PeftModel, PeftConfig

# 构造Peft模型的唯一标识符
# 这里使用模型名称或路径、Peft配置的类型和任务类型来组合成一个字符串
peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"

# 从预训练的Peft模型中加载配置
# 这里的配置将指导如何加载和使用Peft模型
config = PeftConfig.from_pretrained(peft_model_id)
# 从配置中加载基础模型
# 这个基础模型是用于执行序列到序列语言模型任务的
model = AutoModelForSequenceClassification.from_pretrained(config.base_model_name_or_path)
# 使用Peft模型对基础模型进行微调
# 这一步是将基础模型与Peft模型的特定微调参数结合起来
model = PeftModel.from_pretrained(model, peft_model_id)

In [ ]:
# 设置模型为评估模式，以禁用dropout等训练时的行为
model.set_train(False)
# 从验证数据集中获取一个样本，以便后续进行推理演示
example = next(datasets['validation'].create_dict_iterator(output_numpy=True))

# 打印样本中的文本标签，以便用户了解正在处理的数据
print(example['idx'])
# 使用tokenizer对样本文本进行编码，以将其转换为模型可处理的输入格式
inputs = tokenizer(example['idx'], return_tensors="ms")
# 打印编码后的输入，以展示输入格式和内容
print(inputs)

# 禁用梯度计算，以减少计算资源消耗，因为推理过程不需要反向传播
with mindspore._no_grad():
    # 使用模型生成文本，指定最大新生成的令牌数量以限制输出长度
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    # 打印生成的文本编码，以展示模型的输出内容
    print(outputs)
    # 将生成的文本编码转换回人类可读的文本格式，并打印
    print(tokenizer.batch_decode(outputs.asnumpy(), skip_special_tokens=True))